In [1]:
import numpy as np
import os 
import pandas as pd

Valores importantes

K = numero de clusters
p = numero de matrizes de dissimilaridade
n = numero de exemplos


#Dimensionalidades:
 - G = p x K  
 - U = K x N  
 - A = p x K
-  D = p x n x n 

term_1  = K x n
term_2  = x 

# Loading

In [2]:
store = pd.HDFStore("diss_matrices_.h5")

In [3]:
%%time
D_ = [np.expand_dims(store[matrix].to_numpy(), axis=0) for matrix in store]
D_ = np.concatenate(D_,axis=0)
print(D_.shape)


(3, 20867, 20867)
CPU times: user 3.88 s, sys: 9.76 s, total: 13.6 s
Wall time: 47.6 s


In [4]:
store.close()

In [5]:
D_.shape

(3, 20867, 20867)

# Techiniques

In [6]:
def select_medoids_dist(D,G):
    result = np.stack([D[i,G[i],:] for i in range(D.shape[0])], axis=0 )
    return result


In [7]:
def compute_J(G,U,D,A,m=2):
    Dt = select_medoids_dist(D,G)
    Ut = np.power(U,m)
    A = np.einsum('pk,pkn->kn',A,Dt) 
    return np.einsum('kn,kn->',A,Ut)

In [8]:
def compute_U_fuzzy_part(G, U, D, A, m):
    Dk = select_medoids_dist(D,G)
    
    term_1  = np.einsum('pk,pkn->kn',A,Dk)
    term_2 = term_1
    term_1[np.where(term_1 == 0)] = 1


    term_1  = np.power(term_1,-1/(m-1))
    term_2  = np.power(term_2,-1/(m-1))
    term_2  = np.einsum('kn->n',term_2)
    Ut = np.divide(term_1, term_2)
   
    return Ut

In [9]:
def search_best_relev_vec(G,U,D,A,m):
    Dt = select_medoids_dist(D,G)

    Ut = np.power(U,m)
    l = np.einsum('kn,pkn->pk',Ut,Dt)

    a = np.prod(l,axis=0)
    a = np.power(a,(1/(D.shape[0])))
    al = np.divide(a,l)
    
    return al

In [10]:
def search_best_medoids(G,U,D,A,m):

    all = []
    for p in range (D.shape[0]):
        al = []
        for k in range(U.shape[0]):
            h = np.sum(np.power(U[k],m)*D[p],axis=1).argmin()
            al.append(h)
        all.append(al)
    
    all = np.asarray(all)
    return all


In [16]:
def mv_algorithm(D, K, m, T, e, seed = 87):

    """
    
    """

    ########## CONSTRAINTS ##############
     
    assert D.shape[1] == D.shape[2]
    assert K > 1 and K < D.shape[1]
    assert e > 0 
    assert m > 1

    ######### INITIALIZATION #############

    p = D.shape[0]
    n = D.shape[1]
  
    rng        = np.random.default_rng(seed=seed)
    G          = np.stack([rng.choice(n,size=(K),replace=False) for _ in range(n)],axis=0) 
    A          = np.ones(shape=(p,K))
    U          = compute_U_fuzzy_part(G,None,D,A,m)

    
   
    cur_u  = compute_J(G,U,D,A,m)
    


    last_u = 0

    ########## TRAINING LOOP ###############
    for t in range(T):
        
        G = search_best_medoids(G,U,D,A,m)
        A = search_best_relev_vec(G,U,D,A,m)
        

        U          = compute_U_fuzzy_part(G,U,D,A,m)
        


    
        print("Epoch {} ---- ".format(t))
        print("Cur j = "+str(cur_u))
        last_u = cur_u
        cur_u = compute_J(G,U,D,A,m)
        
        print("dif = {}".format(np.absolute(cur_u-last_u)))
        if np.absolute(cur_u-last_u) < e:
            print('Converged in {} iterations'.format(t+1))
            return G,U,D,A, cur_u

In [12]:
K = 12
T = 100
e = 1e-10
m = 2

(3, 20867, 20867)


In [18]:
def run_experiments(D_,K=12,T=150,e=1e-10,m=2,runs=5):
    param = dict()
    best_model = 9999999
    seeds = np.random.randint(0,99999,runs)
    for seed in seeds:
        print("seed number {}".format(seed))
        G,U,D,A,cur_u = mv_algorithm(D_,K,m,T,e,seed)
        if cur_u < best_model:
            param['G'] = G
            param['U'] = U
            param['D'] = D
            param['A'] = A
    return param

In [19]:
param =  run_experiments(D_,K=12,T=150,e=1e-10,m=2,runs=5)

seed number 90092
Epoch 0 ---- 
Cur j = 0.3582565978394258
dif = 0.10789893335399753
Epoch 1 ---- 
Cur j = 0.2503576644854283
dif = 0.000255949664081101
Epoch 2 ---- 
Cur j = 0.2501017148213472
dif = 1.1897139629013509e-07
Epoch 3 ---- 
Cur j = 0.2501015958499509
dif = 5.455091933725953e-11
Converged in 4 iterations
seed number 85171
Epoch 0 ---- 
Cur j = 0.3329654624790211
dif = 0.08244945501986939
Epoch 1 ---- 
Cur j = 0.2505160074591517
dif = 0.0002248064158441032
Epoch 2 ---- 
Cur j = 0.2502912010433076
dif = 5.755634318294067e-05
Epoch 3 ---- 
Cur j = 0.25023364470012466
dif = 2.933713388886261e-08
Epoch 4 ---- 
Cur j = 0.25023361536299077
dif = 8.704370557666152e-12
Converged in 5 iterations
seed number 49822
Epoch 0 ---- 
Cur j = 0.3361321489030547
dif = 0.08562822642107581
Epoch 1 ---- 
Cur j = 0.2505039224819789
dif = 0.0003029006837564907
Epoch 2 ---- 
Cur j = 0.2502010217982224
dif = 1.3981575116894795e-05
Epoch 3 ---- 
Cur j = 0.2501870402231055
dif = 1.6675935077259396e-09